In [ ]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gym[atari,accept-rom-license]==0.21.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 2.9 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=371ff12a43eabbcd75bd89a0db697596557936eeb89d9e984be4a23f680bebde
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441028 sha256=d0674bbca43d56c6bd4902970b28f5996500c2f54b957a7d3b9a1664eb95e67c
  Stored in directory: /root/.cache/pip/wheels/51/08/c5/28b973078691a3f8baf99fcaec1ed8f0e05ef6e54d2390212c
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installatio

In [1]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle
import gym
import time
start_time=time.time()

# hyperparameters
H = 800 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
    if render: env.render()

  # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

  # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

  # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('resetting env. episode reward total was {}. running mean: {}'.format(reward_sum, running_reward))
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None
        if running_reward>=-19:
            break
    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print ('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))
end_time=time.time()
td_model=end_time-start_time
print('Time difference for model is %s seconds:' %(td_model))


/var/folders/mh/b7kpn8490nd8w7l69p27clvc0000gn/T/ipykernel_12972/1349688931.py:39: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return I.astype(np.float).ravel()


ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
ep 0: game finished, reward: -1.0
resetting env. episode reward total was -21.0. running mean: -21.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game finished, reward: -1.0
ep 1: game fini

ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
ep 10: game finished, reward: -1.0
resetting env. episode reward total was -21.0. running mean: -20.961369446021934
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: game finished, reward: -1.0
ep 11: ga

ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
ep 20: game finished, reward: -1.0
resetting env. episode reward total was -21.0. running mean: -20.916544358835576
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: game finished, reward: -1.0
ep 21: ga

ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: 1.0 !!!!!!!!
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
ep 30: game finished, reward: -1.0
resetting env. episode reward total was -20.0. running mean: -20.886074240103678
ep 31: game finished, reward: -1.0
ep 31: game finished, reward: -1.0
ep 31: game finished, reward: -1.0
ep 31: game finished, reward: -1.0
ep 31: game finished, reward: -1.0
e

...

ep 3272: game finished, reward: -1.0
resetting env. episode reward total was -19.0. running mean: -19.157408533293527
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: 1.0 !!!!!!!!
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: 1.0 !!!!!!!!
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: 1.0 !!!!!!!!
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: 1.0 !!!!!!!!
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: -1.0
ep 3273: game finished, reward: 1.0 !!!!!!!!
ep 3273: game finished, rewa

ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: 1.0 !!!!!!!!
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
ep 3281: game finished, reward: -1.0
resetting env. episode reward total was -19.0. running mean: -19.107167959399607
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: 1.0 !!!!!!!!
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game finished, reward: -1.0
ep 3282: game f

ep 3290: game finished, reward: 1.0 !!!!!!!!
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: 1.0 !!!!!!!!
ep 3290: game finished, reward: -1.0
ep 3290: game finished, reward: -1.0
resetting env. episode reward total was -19.0. running mean: -19.079149011544352
ep 3291: game finished, reward: -1.0
ep 3291: game finished, reward: -1.0
ep 3291: game finished, reward: -1.0
ep 3291: game f

ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
ep 3298: game finished, reward: -1.0
resetting env. episode reward total was -21.0. running mean: -19.034805736002646
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward: -1.0
ep 3299: game finished, reward:

ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: 1.0 !!!!!!!!
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: 1.0 !!!!!!!!
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: 1.0 !!!!!!!!
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
ep 3307: game finished, reward: -1.0
resetting env. episode reward total was -18.0. running mean: -19.001324009265257
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308: game finished, reward: -1.0
ep 3308